### General idea
1. Clean dataset => dfClean
2. Vectorize words => to probability density
3. Perform logistic regression on vectorized words 
    of scales of reviews 0 (0,.1),1 (.2,.3) ,2 (.4,.5),3 (.6,.7) ,4 (.8,.9, 1) reviews

In [1]:
# Read data set and stop words
import pandas as pd
import re 
import nltk
#nltk.download('stopwords')
df111 = pd.read_csv (r'.\AppReview.csv')
#df=df1.sample(n=1000)
len(df111.index)


111143

In [2]:
df111.groupby("reviewerRating").nunique()

,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
reviewerRating,,,,,,
0.0,18,1,71,1,60,1
0.2,338,7763,9911,1,3857,24
0.4,294,4012,5134,1,2852,17
0.6,329,6965,8883,1,3759,17
0.8,386,14010,17645,1,4963,25
1.0,473,49121,59953,1,5957,41


In [3]:
frames = []
for i in range(0,12,2):
    frames.append(df111[df111["reviewerRating"]==i/10].sample(n=2000, replace=True))

df=pd.concat(frames)
df.groupby("reviewerRating").nunique()
len(df.index)

12000

In [4]:
# Corpus of stop words
from nltk.corpus import stopwords

In [5]:
# processes a review and returns a list of words
def review_to_words(review, string = True, remove_stopwords=True):
    # Remove HTML
    #review_text = BeautifulSoup(review).get_text()
    review_text=review
    # Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # Convert words to lower case and split them
    words = review_text.lower().split()
    # Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    if string:
        return " ".join(words)
    else:
        return words

In [6]:
#Clean up text
#Remove non-ascii text
#Remove all rows missing reviewerName
def fixString(x):
    return x.encode('ascii',errors='ignore')

# df[["reviewText"]]=df[["reviewText"]].apply(lambda x: str(x["reviewText"]).encode('ascii',errors='ignore').decode(), axis=1)
# df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)
df[["reviewText"]]=df[["reviewText"]].apply(lambda x: review_to_words(x["reviewText"]), axis=1)
df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)

dfCleaned=df[df['reviewText'].str.strip().astype(bool)]
dfCleaned=dfCleaned[df['reviewerName'].str.strip().astype(bool)]

#
dfCleaned

,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
99148,2078,A Google User,exactly isuppose aim touch screen galaxy full ...,0.0,30-December-2010,-9.223372e+18
56238,1508,A Google User,totally lame wsdot cameras video still photos ...,0.0,2011/02/20 00:00:00,-9.223372e+18
99185,2078,A Google User,level three weeks bout give full review,0.0,27-December-2010,-9.223372e+18
98690,2078,A Google User,please add levels full review,0.0,26-January-2011,-9.223372e+18
65027,12,A Google User,worked great started connect laptop stinks ful...,0.0,30-January-2011,-9.223372e+18
...,...,...,...,...,...,...
98405,2078,Iftekhar Akoob,great time waster finished game twice stuck le...,1.0,29-March-2013,-9.223372e+18
4929,1092,Jo Poochie,smiles selection books awesome enough reading ...,1.0,2015/03/28 00:00:00,-9.223372e+18
18041,374,JONATHAN CARTER,app full review,1.0,2017/09/28 00:00:00,-9.223372e+18
71848,2131,Jonathan Payn,awesome far best twitter app market like best ...,1.0,12-October-2015,-9.223372e+18


In [7]:
dfCleaned.count()

appID             11940
reviewerName      11940
reviewText        11940
reviewerRating    11940
reviewDate        11940
textAnalytics     11940
dtype: int64

In [8]:
dfCleaned[dfCleaned['reviewerRating'].apply(lambda x: x==1)].count()/dfCleaned.count()

#type(df)


appID             0.16608
reviewerName      0.16608
reviewText        0.16608
reviewerRating    0.16608
reviewDate        0.16608
textAnalytics     0.16608
dtype: float64

In [9]:
dfCleaned[['reviewText']]

,reviewText
99148,exactly isuppose aim touch screen galaxy full ...
56238,totally lame wsdot cameras video still photos ...
99185,level three weeks bout give full review
98690,please add levels full review
65027,worked great started connect laptop stinks ful...
...,...
98405,great time waster finished game twice stuck le...
4929,smiles selection books awesome enough reading ...
18041,app full review
71848,awesome far best twitter app market like best ...


## Vectorize words
### This is based on 
https://towardsdatascience.com/sentiment-analysis-a-how-to-guide-with-movie-reviews-9ae335e6bcb2
With actual logistic regression:
https://towardsdatascience.com/sentiment-classification-with-logistic-regression-analyzing-yelp-reviews-3981678c3b44


In [10]:
# from gensim.sklearn_api import W2VTransformer

# model = W2VTransformer(size=10, min_count=1, seed=1)

In [11]:
# import statements
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import scipy
from sklearn.model_selection import train_test_split

dfReviews = dfCleaned['reviewText']
dfTrain, dfTest = train_test_split(dfReviews, test_size=.1)

# Initialize a bag of words
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 1000) 

# Fit transform the data 
train_feat = vectorizer.fit_transform(dfTrain).toarray()
test_feat = vectorizer.transform(dfTest).toarray()


# TFIDF train set
tfidf_transformer = TfidfTransformer().fit(train_feat)
train_tfidf = tfidf_transformer.transform(train_feat)
 
# apply tfidf to test set
test_tfidf = tfidf_transformer.transform(test_feat)

train_tfidf.todense()


matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.21650386, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.48438822,
         0.        ]])

In [12]:
#rv1_tf.todense()
a=train_tfidf.todense()
print(type(a))
pddf = pd.DataFrame(data=train_tfidf.todense())
print(type(pddf))
print(a)
print(pddf)


<class 'numpy.matrix'>
<class 'pandas.core.frame.DataFrame'>
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.21650386 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.48438822 0.        ]]
       0         1    2    3    4    5    6    7    8    9    ...  990  991  \
0      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4      0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...    ...       ... 

In [13]:
# print(a[99430,998])
# print(pddf[998][99430])

# pd.merge(pddf, dfCleaned[['reviewerRating']], left_index=True, right_index=True)

In [14]:
dfTest
rv1 = pd.Series("bad terrible")
rv1_tf = tfidf_transformer.transform(vectorizer.transform(rv1))
# individual_x=cx_t.todense()
# clf.predict_proba(individual_x)

In [15]:
featureNamesList=vectorizer.get_feature_names()
rv1_tf
cx_t = scipy.sparse.coo_matrix(rv1_tf)


print(rv1)
for i,j,v in zip(cx_t.row, cx_t.col, cx_t.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))

0    bad terrible
dtype: object
(0, 860), terrible = 0.7750305703872686
(0, 60), bad = 0.6319237414159916


In [16]:
# individual_x=cx_t.todense()
# clf.predict(individual_x)


In [17]:
pddf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10741,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
10742,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
10743,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
10744,0.0,0.216504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [33]:
#train data
trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
#determined feature names
featureNamesList=vectorizer.get_feature_names()

trainYdata
dfCleaned[['reviewerRating']],dfTrain.to_frame()

(       reviewerRating
 99148             0.0
 56238             0.0
 99185             0.0
 98690             0.0
 65027             0.0
 ...               ...
 98405             1.0
 4929              1.0
 18041             1.0
 71848             1.0
 63511             1.0
 
 [11940 rows x 1 columns],
                                                reviewText
 109560  used work gracefully uploading picture pain le...
 109231  app make post sticky button would perfect woul...
 94716             keep server settings galaxy full review
 71616   best fav star button fav star button automatic...
 252     fast smooth provides fastest smoothest browser...
 ...                                                   ...
 12341   essential app us security wise folks good job ...
 38514                                         full review
 13642   mr lodu aap kya karna kuch samaj hi nai aa ra ...
 92816   workaround folder problem workaround able see ...
 18619   trash external links crappy youtube v

In [19]:
# look at data for training
type(dfTrain)
stopcounter = 0
for index, value in dfTrain.items():
    print(f"Index : {index}, Value : {value}")
    stopcounter=stopcounter+1
    if stopcounter > 10:
        break

Index : 109560, Value : used work gracefully uploading picture pain let change picture size like really disappointed full review
Index : 109231, Value : app make post sticky button would perfect would never need log web interface sticky posts use lot theme full review
Index : 94716, Value : keep server settings galaxy full review
Index : 71616, Value : best fav star button fav star button automatically colors yellow though im clicking doesnt count annoying please take action reallylove app full review
Index : 252, Value : fast smooth provides fastest smoothest browser experience full review
Index : 7843, Value : recognise player rooted epic stock player full review
Index : 101507, Value : ppl cant beet tip keep middle open n stop complaining full review
Index : 99148, Value : exactly isuppose aim touch screen galaxy full review
Index : 7960, Value : doubletwist support full review
Index : 21216, Value : installing asus zenfone full review
Index : 11400, Value : great app cross platform

In [20]:
type(train_feat)
#len(train_feat)

numpy.ndarray

In [21]:
# Dump some vectorized words and probablities
import scipy.sparse

featureNamesList=vectorizer.get_feature_names()
type(dfTrain)

cx = scipy.sparse.coo_matrix(train_feat)
cx2 = scipy.sparse.coo_matrix(train_tfidf)


print(dfTrain.iloc[0])
for i,j,v in zip(cx.row, cx.col, cx.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))
    
print("\r\n\r\nThe weighted results\r\n")
for i,j,v in zip(cx2.row, cx2.col, cx2.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))

used work gracefully uploading picture pain let change picture size like really disappointed full review
(0, 113), change = 1
(0, 218), disappointed = 1
(0, 328), full = 1
(0, 467), let = 1
(0, 474), like = 1
(0, 629), picture = 2
(0, 682), really = 1
(0, 710), review = 1
(0, 779), size = 1
(0, 927), used = 1
(0, 980), work = 1


The weighted results

(0, 980), work = 0.17681132354250811
(0, 927), used = 0.23711341310940923
(0, 779), size = 0.3463614324765156
(0, 710), review = 0.054540329467902895
(0, 682), really = 0.22174875297030228
(0, 629), picture = 0.624869103742966
(0, 474), like = 0.19952579795456063
(0, 467), let = 0.3039451690770323
(0, 328), full = 0.054540329467902895
(0, 218), disappointed = 0.3717821199082091
(0, 113), change = 0.276677406761197


In [22]:
# look at some more data
import scipy.sparse

featureNamesList=vectorizer.get_feature_names()
type(test_tfidf)

cx = scipy.sparse.coo_matrix(test_tfidf)

print(dfTest.iloc[0])
for i,j,v in zip(cx.row, cx.col, cx.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))


filtering apps based already icons makes pain use really save everyone time put apps already numix icons list full review
(0, 926), use = 0.14045856328170228
(0, 874), time = 0.15046358618744912
(0, 724), save = 0.21547102265884396
(0, 710), review = 0.03897253378963685
(0, 682), really = 0.15845358567243056
(0, 669), put = 0.22536149257994215
(0, 514), makes = 0.20442767150945865
(0, 480), list = 0.21614882293934295
(0, 396), icons = 0.5475083991717641
(0, 328), full = 0.03897253378963685
(0, 265), everyone = 0.2722833633783385
(0, 44), apps = 0.36142621405564296
(0, 27), already = 0.48910909807203506


### Try svm

In [37]:
# from sklearn import svm

In [18]:
# # Get trained Y data and test Y data
# trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
# testYdata = pd.merge(dfTest.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
# # Get trained X data and test X data
# trainXdata = train_tfidf.todense()
# testXdata = test_tfidf.todense()

In [23]:
# # Train data - map the y to ints of scales of reviews 0,1,2,3,4 reviews
# y = trainYdata[['reviewerRating']]
# #y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
# y_int = trainYdata['reviewerRating'].apply(lambda x: x*10)
# #y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x <.5 else 1)
# y_int_test = testYdata['reviewerRating'].apply(lambda x: x*10)
# X = trainXdata

In [24]:
# clf = svm.SVC()
# clf.fit(X,y_int)



In [25]:
# clf.predict(X[:2, :])
# clf.score(X, y_int)


# clf.predict(testXdata)
# clf.score(testXdata, y_int_test)

### Modelling part
1. Leverage the raw vector count and the tf-idf weighted version

In [26]:
# # Just looking at some data
# train_tfidf
# dfTrain
# print(df.iloc[105146])
# dfTrain
# df
# train_tfidf.todense()
# dfTrain.to_frame()

In [27]:
dfTest.head(10)
#type(dfTest)

78652     filtering apps based already icons makes pain ...
106194    worst app try altough best app pc tab cant pla...
47311                      ni whois user screen full review
94483                                   awesome full review
68271                                  good app full review
61135     need updating though bc lot stuff published pa...
94041     work sick rage using sickbeard worked fine mov...
96251     stopped work today update claims keys secure s...
1030      beautiful clock problem follow actual time ful...
87194     works great feature bloat best feature one thi...
Name: reviewText, dtype: object

In [28]:
# Get trained Y data and test Y data
trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
testYdata = pd.merge(dfTest.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)

In [29]:
trainYdata

,reviewText,reviewerRating
10,solid lightweight browser ad blocking basic fe...,0.8
28,download images anymore please fix full review,0.4
30,easy way switch mobile desktop site full review,0.2
35,stay frozen continuously continue improving fu...,0.6
39,like chrome advertisements makes using chome u...,0.8
...,...,...
111134,battery super use full review,0.8
111134,battery super use full review,0.8
111134,battery super use full review,0.8
111134,battery super use full review,0.8


In [70]:
# Get trained X data and test X data
trainXdata = train_tfidf.todense()
testXdata = test_tfidf.todense()


In [44]:
print(dfCleaned[20520:20521])
print(df1[20624:20625])
print(testYdata[0:1])
print(testXdata[0:1])

Empty DataFrame
Columns: [appID, reviewerName, reviewText, reviewerRating, reviewDate, textAnalytics]
Index: []
       appID reviewerName                                         reviewText  \
20624   1410    Edgar Lim  Nice work! I have purchased the unlocked featu...   

       reviewerRating           reviewDate  textAnalytics  
20624             1.0  2016/08/14 00:00:00            NaN  
                                            reviewText  reviewerRating
152  latest version buggy latest version fails open...             1.0
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.55801087 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        

In [73]:
len(trainYdata.index)

62643

In [45]:
#from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [71]:
# Train data - map the y to ints of scales of reviews 0,1,2,3,4 reviews
y = trainYdata[['reviewerRating']]
#y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
y_int = trainYdata['reviewerRating'].apply(lambda x: x*10)
#y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x <.5 else 1)

X = trainXdata

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X, y_int)

# Check trained accuracy
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y_int)


ValueError: Found input variables with inconsistent numbers of samples: [10764, 62643]

In [46]:
clf.predict_proba(X[:2, :])

array([[2.53423095e-04, 1.08950810e-03, 7.45332417e-03, 9.20661122e-02,
        6.90839347e-01, 2.08298286e-01],
       [1.29446140e-04, 2.31550156e-03, 1.10806381e-03, 4.35405832e-03,
        6.45745041e-02, 9.27518426e-01]])

In [47]:
X[:2, :]

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

### Check test accuracy

In [48]:
#y_int_test = testYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
y_int_test = testYdata['reviewerRating'].apply(lambda x: x*10)
#y_int_test = testYdata['reviewerRating'].apply(lambda x: 0 if x<.5 else 1)

In [49]:
clf.predict(testXdata)
clf.predict_proba(testXdata)
clf.score(testXdata, y_int_test)

0.6753258508327299

In [50]:
testXdata

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 10, n_jobs=-1)
# Train the model on training data
clt = rf.fit(X, y_int);

In [52]:
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y_int)


clf.predict(testXdata)
clf.predict_proba(testXdata)
clf.score(testXdata, y_int_test)

0.6753258508327299